In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [21]:
DOU_ED_Volume = pd.read_csv("DOU CY2020 ED Patient Volume.csv")
Q1 = DOU_ED_Volume[DOU_ED_Volume.Month.isin([1,2,3])]
Q2 = DOU_ED_Volume[DOU_ED_Volume.Month.isin([4,5,6])]
Q3 = DOU_ED_Volume[DOU_ED_Volume.Month.isin([7,8,9])]
Q4 = DOU_ED_Volume[DOU_ED_Volume.Month.isin([10,11,12])]

In [11]:
ED_Volume_Data = pd.read_csv("2021_Q2 DOU ED Patient Volume.csv")

In [12]:
ED_Volume_Data

,Hospital,UnitName,Month,NumberofPsychiatricPatientsAdmitted,NumberofEDObservationPatientsAdmitted,NumberofAllOtherPatientsAdmitted,NumberofPsychiatricPatientsTransferred,NumberofEDObservationPatientsTransferred,NumberofAllOtherPatientsTransferred,NumberofPsychiatricPatientsDischarged,NumberofEDObservationPatientsDischarged,NumberofAllOtherPatientsDischarged
0,EHH,EHH ED,6,0,12,361,0,0,61,0,0,4218
1,EDH,EDH ED,4,69,22,756,9,1,155,3,0,3569
2,EHH,EHH ED,5,0,17,361,0,0,47,0,0,4066
3,EDH,EDH ED,5,77,18,737,8,0,146,2,0,3801
4,EHH,EHH ED,4,0,11,389,0,2,69,0,0,4084
5,EDH,EDH ED,6,85,11,679,6,2,144,5,0,3852


In [6]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_EDVolume_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    
    # The first step takes only the data for this selected hospital
    Hosp_data = ED_Volume_Data[ED_Volume_Data.Hospital == Selected_Hosp]
    # Now this is a List of this Hospital's Units
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is the xml's entire UnitYrQtr node
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter
    

    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with the months from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            # This is looping through the 'xPatientDaysMethodFID', 'PtDaysMNCensus' columns from the data
            for column in new_data.columns[3:]:

                #Lastly it is replacing the values in the 'XML Template' with the correct values from the data.
                new_value = new_data[column].iloc[idx2]
                month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} ED_Patient_Volume.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUH

In [7]:
NDNQI_EDVolume_XML('EUH','2020 ED Pt Volume Template.xml','JXE432',2021,2)

## EUHM

In [8]:
NDNQI_EDVolume_XML('EUHM','2020 ED Pt Volume Template.xml','PVF155',2021,2)

## ESJH

In [9]:
NDNQI_EDVolume_XML('ESJH','2020 ED Pt Volume Template.xml','XAR317',2021,2)

## EJCH

In [10]:
NDNQI_EDVolume_XML('EJCH','2020 ED Pt Volume Template.xml','YTD069',2021,2)

## EDH

In [13]:
NDNQI_EDVolume_XML('EDH','2021 DOU ED Pt Volume Template.xml','NMW346',2021,2)

## EHH

In [14]:
NDNQI_EDVolume_XML('EHH','2021 DOU ED Pt Volume Template.xml','WXX116',2021,2)